# 🚀 Operational Flow:

**Executes the entire flow with a notebook interface:**
- Initiate DB
- Load CSV to DB
- Load (and validate) TAK repository
- Apply TAKs on DB
- View output

In [ ]:
# !pip install -r requirements-py37.txt
# !pip install -e .

In [12]:
from pathlib import Path
from backend.dataaccess import DataAccess
from core.mediator import Mediator, setup_logging
import logging
import pandas as pd
from collections import defaultdict

In [13]:
# Paths
KB_PATH = Path("core/knowledge-base")
DB_PATH = Path("backend/data/mediator.db")
CSV_PATH = Path("backend/data/input_data.csv")

In [14]:
# # 1. Connect to existing DB
# da = DataAccess(db_path=str(DB_PATH))

# 2. Or auto-create (and optionally drop existing)
da = DataAccess(db_path=str(DB_PATH), auto_create=True)

# Check stats
stats = da.get_table_stats()
for table, info in stats.items():
    print(f"{table}: {info['rows']} rows, {info['n_patients']} patients")

[Info] Keeping existing database structure.
InputPatientData: 1790 rows, 12 patients
OutputPatientData: 2990 rows, 12 patients
PatientQAScores: 1036 rows, 12 patients


In [15]:
# Load CSV into InputPatientData
total_rows = da.load_csv_to_input(
    csv_path=str(CSV_PATH),
    if_exists='append',           # 'append' or 'replace'
    clear_output_and_qa=False,    # Set True to clear outputs
    yes=True                      # Auto-confirm
)
print(f"Loaded {total_rows} rows")

[Info] Validating+inserting CSV in chunks (pandas)...


CSV chunks: 0chunk [00:00, ?chunk/s]

CSV chunks: 1chunk [00:00, 28.38chunk/s]

[Info] Finished loading. Inserted 0 rows.
[Info]: DB initiated successfully!
[Info]: Total tables created: 4
[Info]: Table 'sqlite_sequence' - Rows: 3
[Info]: Table 'InputPatientData' - Rows: 1790
[Info]: Table 'OutputPatientData' - Rows: 2990
[Info]: Table 'PatientQAScores' - Rows: 1036
Loaded 0 rows


In [16]:
# Setup logging ONCE (before creating Mediator)
setup_logging(
    log_file=Path("logs/mediator.log"),
    console_level=logging.WARNING  # Only warnings/errors in console
)

# Initialize mediator
mediator = Mediator(knowledge_base_path=KB_PATH, data_access=da)

# Build TAK repository
repo = mediator.build_repository()

# List all TAK names
print("TAK Names:")
for tak_name in sorted(repo.taks.keys()):
    print(f"  - {tak_name}")

# Save repository to file (uncompressed)
repo.save("backend/data/tak-repo.pkl", compress=False)


PHASE 1: Building TAK Repository


Loading TAKs:   0%|          | 0/163 [00:00<?, ?file/s, Raw Concepts: ACUTE_RESPIRATORY_DISORDER]

Loading TAKs: 100%|██████████| 163/163 [00:00<00:00, 258.15file/s, Global Patterns: ROUTINE_GLUCOSE_MEASURE_PATTERN]       



[Validation] Running business-logic checks on TAK repository...

✅ TAK Repository Built Successfully
  Raw Concepts: 71
  Events:       25
  States:       27
  Trends:       24
  Contexts:     5
  Patterns:     11
  TOTAL TAKs:   163

TAK Names:
  - ACIDOSIS_EVENT
  - ACUTE_RESPIRATORY_DISORDER
  - ACUTE_RESPIRATORY_DISORDER_EVENT
  - ADMISSION
  - ADMISSION_EVENT
  - ALANINE-AMINOTRANSFERASE_MEASURE
  - ALANINE-AMINOTRANSFERASE_MEASURE_STATE
  - ALANINE-AMINOTRANSFERASE_MEASURE_TREND
  - ALBUMIN_MEASURE
  - ALBUMIN_MEASURE_STATE
  - ALBUMIN_MEASURE_TREND
  - ANTIBIOTIC_BITZUA_EVENT
  - ANTIBIOTIC_IV_BITZUA
  - ANTIBIOTIC_PO_BITZUA
  - ANTIDIABETIC_BITZUA
  - ASPARATE-AMINOTRANSFERASE_MEASURE
  - ASPARATE-AMINOTRANSFERASE_MEASURE_STATE
  - ASPARATE-AMINOTRANSFERASE_MEASURE_TREND
  - ATHEROSCLEROSIS
  - ATHEROSCLEROSIS_EVENT
  - BASAL_BITZUA
  - BASAL_BITZUA_RATIO
  - BASAL_BITZUA_STATE
  - BASE_GLUCOSE_MEASURE
  - BICARBONATE_BITZUA
  - BICARBONATE_BITZUA_EVENT
  - BICARBONATE_MEASURE

In [ ]:
# # Process specific patients (Jupyter-compatible)
# patient_ids = [1000]
patient_ids = None
patient_stats = mediator.run(
        max_concurrent=4,
        patient_subset=patient_ids
)

# Ensure required objects exist
assert 'patient_stats' in globals(), "patient_stats not found. Run the processing cell first."
assert 'repo' in globals(), "repo not found. Build the TAK repository first."

all_tak_names = sorted(repo.taks.keys())
agg_counts = defaultdict(int)
errors = []

for pid, stats in patient_stats.items():
    if not isinstance(stats, dict):
        continue
    # Top-level pipeline error
    if 'error' in stats and isinstance(stats['error'], str):
        errors.append((pid, '<pipeline>', stats['error']))
    for tak_name, val in stats.items():
        if tak_name == 'error':
            continue
        if isinstance(val, int):
            agg_counts[tak_name] += val
        elif isinstance(val, str) and val.startswith("ERROR"):
            errors.append((pid, tak_name, val))

# Build usage DataFrame
df_usage = pd.DataFrame(
    [(tak, int(agg_counts.get(tak, 0))) for tak in all_tak_names],
    columns=["TAK", "TotalRows"]
)

df_usage_sorted = df_usage.sort_values(by="TotalRows", ascending=False, ignore_index=True)

unused_taks = [tak for tak in all_tak_names if agg_counts.get(tak, 0) == 0]

# Summary (avoid surrogate escapes; use literal emoji or plain text)
print("\n" + "="*80)
print("📊 Aggregate TAK Usage Across Patients")
print("="*80)

total_rows_all = int(df_usage["TotalRows"].sum())
print(f"Total rows calculated: {total_rows_all} (including Raw Concepts)")
print(f"Total TAKs: {len(all_tak_names)}")
print(f"Used TAKs: {len(all_tak_names) - len(unused_taks)}")
print(f"Unused TAKs: {len(unused_taks)}")

# Top-N TAKs
top_n = 15
print(f"\n🏆 Top {top_n} TAKs by rows:")
for _, row in df_usage_sorted.head(top_n).iterrows():
    print(f"  • {row['TAK']}: {int(row['TotalRows'])} rows")

# Unused TAKs
if unused_taks:
    print("\n⚪ Unused TAKs (0 rows across all patients):")
    for i in range(0, len(unused_taks), 3):
        print("  " + ", ".join(unused_taks[i:i+3]))

# Errors grouped by TAK
if errors:
    print("\n❌ Errors by TAK (with patient IDs):")
    errs_by_tak = defaultdict(list)
    for pid, tak, msg in errors:
        errs_by_tak[tak].append((pid, msg))
    for tak, items in errs_by_tak.items():
        print(f"  • {tak}: {len(items)} errors")
        for pid, msg in items[:5]:
            print(f"      - patient {pid}: {msg}")

# Display full usage table
display(df_usage_sorted)


PHASE 1: Building TAK Repository


Loading TAKs:   0%|          | 0/163 [00:00<?, ?file/s]

Loading TAKs: 100%|██████████| 163/163 [00:00<00:00, 235.75file/s, Global Patterns: ROUTINE_GLUCOSE_MEASURE_PATTERN]       



[Validation] Running business-logic checks on TAK repository...

✅ TAK Repository Built Successfully
  Raw Concepts: 71
  Events:       25
  States:       27
  Trends:       24
  Contexts:     5
  Patterns:     11
  TOTAL TAKs:   163


PHASE 2: Processing 12 Patients (max_workers=4)



Processing patients: 100%|██████████| 12/12 [00:06<00:00,  1.73patient/s]


✅ Patient Processing Complete
  Patients processed: 12
  Total rows written: 5241
  Errors:             0


📊 Aggregate TAK Usage Across Patients
Total rows written: 5241
Total TAKs: 163
Used TAKs: 34
Unused TAKs: 129

🏆 Top 15 TAKs by rows:
  • GLUCOSE_MEASURE: 696 rows
  • NORMAL_GLUCOSE_IND: 445 rows
  • GLUCOSE_MEASURE_TREND: 378 rows
  • GLUCOSE_MEASURE_ON_INSULIN_PATTERN: 344 rows
  • INSULIN_BITZUA_CONTEXT: 344 rows
  • GLUCOSE_MEASURE_STATE: 329 rows
  • MEAL: 302 rows
  • MEAL_CONTEXT: 302 rows
  • BOLUS_BITZUA: 248 rows
  • DISGLYCEMIA_EVENT: 245 rows
  • BOLUS_BITZUA_STATE: 202 rows
  • HIGH_GLUCOSE_IND: 179 rows
  • GLUCOSE_MEASURE_ON_HYPER_PATTERN: 177 rows
  • BASAL_BITZUA_RATIO: 168 rows
  • HIGH_GLUCOSE_CONTEXT: 156 rows

⚪ Unused TAKs (0 rows across all patients):
  ACIDOSIS_EVENT, ACUTE_RESPIRATORY_DISORDER, ACUTE_RESPIRATORY_DISORDER_EVENT
  ALANINE-AMINOTRANSFERASE_MEASURE, ALANINE-AMINOTRANSFERASE_MEASURE_STATE, ALANINE-AMINOTRANSFERASE_MEASURE_TREND
  ALBUMIN_MEA

,TAK,TotalRows
0,GLUCOSE_MEASURE,696
1,NORMAL_GLUCOSE_IND,445
2,GLUCOSE_MEASURE_TREND,378
3,GLUCOSE_MEASURE_ON_INSULIN_PATTERN,344
4,INSULIN_BITZUA_CONTEXT,344
...,...,...
158,TROPONIN_MEASURE_TREND,0
159,UREA_MEASURE_STATE,0
160,UREA_MEASURE_TREND,0
161,URINE_CULTURE,0


In [11]:
# Query OutputPatientData
query = """
SELECT PatientId, ConceptName, StartDateTime, EndDateTime, Value
FROM OutputPatientData
WHERE PatientId = 1000
"""

df_results = pd.read_sql_query(query, da.conn)
df_results = df_results.sort_values(by=["StartDateTime"], ignore_index=True)
df_results.head(20)

,PatientId,ConceptName,StartDateTime,EndDateTime,Value
0,1000,ADMISSION_EVENT,2025-01-21 09:00:00,2025-01-21 09:00:01,True
1,1000,INSULIN_ON_ADMISSION_PATTERN,2025-01-21 09:00:00,2025-01-21 13:07:01,Partial
2,1000,GLUCOSE_MEASURE_ON_ADMISSION_PATTERN,2025-01-21 09:00:00,2025-01-21 12:00:01,True
3,1000,DIABETES_DIAGNOSIS_CONTEXT,2025-01-21 09:00:00,2025-01-24 14:00:01,True
4,1000,STOP_ANTIDIABETICS_ON_ADMISSION_PATTERN,2025-01-21 09:00:00,2025-01-21 09:00:01,False
5,1000,BMI_MEASURE_ON_ADMISSION,2025-01-21 09:00:00,2025-01-21 15:00:01,True
6,1000,MEAL_CONTEXT,2025-01-21 11:00:00,2025-01-21 15:00:01,Lunch
7,1000,GLUCOSE_MEASURE_STATE,2025-01-21 12:00:00,2025-01-21 16:00:00,Normal Glucose
8,1000,ROUTINE_GLUCOSE_MEASURE_PATTERN,2025-01-21 12:00:00,2025-01-22 12:00:00,Partial
9,1000,GLUCOSE_MEASURE_TREND,2025-01-21 12:00:00,2025-01-21 16:00:00,Increasing


In [9]:
# Query OutputPatientData
query = """
SELECT PatientId, PatternName, StartDateTime, EndDateTime, ComplianceType, ComplianceScore
FROM PatientQAScores
WHERE PatientId = 1000
"""
df_results = pd.read_sql_query(query, da.conn)
df_results = df_results.sort_values(by=["PatternName"], ignore_index=True)
df_results

,PatientId,PatternName,StartDateTime,EndDateTime,ComplianceType,ComplianceScore
0,1000,BMI_MEASURE_ON_ADMISSION,2025-01-21 09:00:00,2025-01-21 15:00:01,TimeConstraint,1.000000
1,1000,CREATININE_MEASURE_ON_ADMISSION,2262-04-11 23:47:16.854775807,2262-04-11 23:47:16.854775807,TimeConstraint,0.000000
2,1000,GLUCOSE_MEASURE_ON_ADMISSION_PATTERN,2025-01-21 09:00:00,2025-01-21 12:00:01,TimeConstraint,1.000000
3,1000,GLUCOSE_MEASURE_ON_HYPER_PATTERN,2025-01-21 16:00:00,2025-01-21 20:00:01,TimeConstraint,1.000000
4,1000,GLUCOSE_MEASURE_ON_HYPER_PATTERN,2025-01-22 00:00:00,2025-01-22 04:00:01,TimeConstraint,1.000000
5,1000,GLUCOSE_MEASURE_ON_HYPER_PATTERN,2025-01-23 00:00:00,2025-01-23 04:00:01,TimeConstraint,1.000000
6,1000,GLUCOSE_MEASURE_ON_INSULIN_PATTERN,2025-01-24 12:51:00,2025-01-24 12:51:01,ValueConstraint,0.000000
7,1000,GLUCOSE_MEASURE_ON_INSULIN_PATTERN,2025-01-24 12:51:00,2025-01-24 12:51:01,TimeConstraint,0.000000
8,1000,GLUCOSE_MEASURE_ON_INSULIN_PATTERN,2025-01-24 08:16:00,2025-01-24 12:00:01,TimeConstraint,1.000000
9,1000,GLUCOSE_MEASURE_ON_INSULIN_PATTERN,2025-01-23 21:00:00,2025-01-24 00:00:01,TimeConstraint,1.000000


# 🔍 TAK Debugging Toolkit

Interactive debugging for a single patient's abstraction flow.

**Features:**
- Load minimal data (only concepts relevant to selected TAKs)
- Use isolated debug DB (no production contamination)
- Full TAK execution trace with cache inspection
- Compare input → intermediate → output at each layer

In [12]:
# ============================================================
# Configuration: Select Patient & TAKs to Debug
# ============================================================

DEBUG_PATIENT_ID = 1000

# Select TAKs to trace (in dependency order)
DEBUG_TAKS = [
    # Raw concepts
    "DIABETES_DIAGNOSIS",
    "ADMISSION",
    "GLUCOSE_MEASURE",
    "BASAL_BITZUA",
    "DEATH",
    "RELEASE",
    
    # Events
    "ADMISSION_EVENT",
    "DEATH_EVENT",
    "RELEASE_EVENT",
    
    # Contexts
    "DIABETES_DIAGNOSIS_CONTEXT",
    
    # Patterns
    "GLUCOSE_MEASURE_ON_ADMISSION_PATTERN"
]

# Debug DB path (separate from production)
DEBUG_DB_PATH = Path("backend/data/debug_mediator.db")

In [13]:
# ============================================================
# Step 1: Extract Minimal Input Data for Debug Patient
# ============================================================

# Query production DB for this patient's input data
query = f"""
SELECT DISTINCT PatientId, ConceptName, StartDateTime, EndDateTime, Value
FROM InputPatientData
WHERE PatientId = {DEBUG_PATIENT_ID}
ORDER BY StartDateTime, ConceptName
"""

df_patient_input = pd.read_sql_query(query, da.conn)

print(f"📥 Input data for patient {DEBUG_PATIENT_ID}:")
print(f"   Total rows: {len(df_patient_input)}")
print(f"\n📊 Unique concepts in input:")
for concept in sorted(df_patient_input['ConceptName'].unique()):
    count = (df_patient_input['ConceptName'] == concept).sum()
    print(f"   - {concept}: {count} rows")

# Show sample
display(df_patient_input.head(10))

📥 Input data for patient 1000:
   Total rows: 63

📊 Unique concepts in input:
   - ADMISSION: 1 rows
   - BASAL_DOSAGE: 3 rows
   - BASAL_ROUTE: 3 rows
   - BMI_MEASURE: 1 rows
   - BOLUS_DOSAGE: 10 rows
   - BOLUS_ROUTE: 10 rows
   - DIABETES_DIAGNOSIS: 1 rows
   - GLUCOSE_MEASURE: 22 rows
   - MEAL: 10 rows
   - RELEASE: 1 rows
   - WEIGHT_MEASURE: 1 rows


,PatientId,ConceptName,StartDateTime,EndDateTime,Value
0,1000,ADMISSION,2025-01-21 09:00:00,2025-01-21 09:00:01,True
1,1000,DIABETES_DIAGNOSIS,2025-01-21 09:00:00,2025-01-21 09:00:01,True
2,1000,GLUCOSE_MEASURE,2025-01-21 12:00:00,2025-01-21 12:00:01,172.4
3,1000,MEAL,2025-01-21 13:00:00,2025-01-21 13:00:01,Lunch
4,1000,BOLUS_DOSAGE,2025-01-21 13:07:00,2025-01-21 13:07:01,14.6
5,1000,BOLUS_ROUTE,2025-01-21 13:07:00,2025-01-21 13:07:01,IntraVenous
6,1000,BMI_MEASURE,2025-01-21 15:00:00,2025-01-21 15:00:01,25.3
7,1000,WEIGHT_MEASURE,2025-01-21 15:00:00,2025-01-21 15:00:01,83.9
8,1000,GLUCOSE_MEASURE,2025-01-21 16:00:00,2025-01-21 16:00:01,216.2
9,1000,MEAL,2025-01-21 19:00:00,2025-01-21 19:00:01,Dinner


In [14]:
# ============================================================
# Step 2: Create Debug DB and Load Minimal Data
# ============================================================

# Close any existing debug DB connections
if 'debug_da' in globals() and hasattr(debug_da, 'conn'):
    try:
        debug_da.conn.close()
        del debug_da
    except:
        pass

# Delete debug DB file entirely (force fresh start)
if DEBUG_DB_PATH.exists():
    import time
    for _ in range(3):  # Retry up to 3 times
        try:
            DEBUG_DB_PATH.unlink()
            print(f"🗑️  Deleted old debug DB: {DEBUG_DB_PATH}")
            break
        except PermissionError:
            time.sleep(0.5)  # Wait for file handles to release
    else:
        print(f"⚠️  Could not delete debug DB (file in use). Restart kernel and try again.")
        raise

# Create fresh debug DB
debug_da = DataAccess(db_path=str(DEBUG_DB_PATH), auto_create=True)
print(f"✅ Created fresh debug DB at {DEBUG_DB_PATH}")

# Load ONLY concepts relevant to DEBUG_TAKS
# Build list of input concepts needed by selected TAKs
mediator_temp = Mediator(knowledge_base_path=KB_PATH, data_access=debug_da)
repo_temp = mediator_temp.build_repository()

needed_concepts = set()
for tak_name in DEBUG_TAKS:
    tak = repo_temp.get(tak_name)
    if tak:
        if hasattr(tak, 'derived_from'):
            if isinstance(tak.derived_from, list):
                # Event/Context/Pattern
                for df in tak.derived_from:
                    needed_concepts.add(df['name'])
            else:
                # State/Trend
                needed_concepts.add(tak.derived_from)
        
        # Add attributes for raw-concepts
        if hasattr(tak, 'attributes'):
            for attr in tak.attributes:
                needed_concepts.add(attr['name'])

print(f"\n🎯 Concepts needed for selected TAKs:")
print(f"   {sorted(needed_concepts)}")

# Filter input data to only needed concepts
df_filtered = df_patient_input[df_patient_input['ConceptName'].isin(needed_concepts)].copy()

print(f"\n📦 Filtered input data:")
print(f"   Rows: {len(df_patient_input)} → {len(df_filtered)}")
print(f"   Concepts: {df_patient_input['ConceptName'].nunique()} → {df_filtered['ConceptName'].nunique()}")

# Load to debug DB
if not df_filtered.empty:
    df_filtered.to_sql('InputPatientData', debug_da.conn, if_exists='append', index=False)
    print(f"\n✅ Loaded {len(df_filtered)} rows to debug DB")
else:
    print(f"\n⚠️  No data to load (needed concepts not found in input)")

🗑️  Deleted old debug DB: backend\data\debug_mediator.db
[Info]: Initializing new database at backend\data\debug_mediator.db
[Info] Dropping existing tables...
[Info] Creating tables from DDL...
[Info]: DB initiated successfully!
[Info]: Total tables created: 4
[Info]: Table 'InputPatientData' - Rows: 0
[Info]: Table 'sqlite_sequence' - Rows: 0
[Info]: Table 'OutputPatientData' - Rows: 0
[Info]: Table 'PatientQAScores' - Rows: 0
✅ Created fresh debug DB at backend\data\debug_mediator.db

PHASE 1: Building TAK Repository


Loading TAKs:   8%|▊         | 3/37 [00:00<00:00, 210.54file/s, Raw Concepts: BMI_MEASURE]                 

Loading TAKs: 100%|██████████| 37/37 [00:00<00:00, 249.90file/s, Patterns: INSULIN_ON_HIGH_GLUCOSE_PATTERN]     


[Validation] Running business-logic checks on TAK repository...

✅ TAK Repository Built Successfully
  Raw Concepts: 15
  Events:       4
  States:       5
  Trends:       2
  Contexts:     6
  Patterns:     5
  TOTAL TAKs:   37


🎯 Concepts needed for selected TAKs:
   ['ADMISSION', 'ADMISSION_EVENT', 'BASAL_DOSAGE', 'BASAL_ROUTE', 'DEATH', 'DIABETES_DIAGNOSIS', 'DIABETES_DIAGNOSIS_CONTEXT', 'GLUCOSE_MEASURE', 'RELEASE']

📦 Filtered input data:
   Rows: 63 → 31
   Concepts: 11 → 6

✅ Loaded 31 rows to debug DB


In [15]:
# ============================================================
# Step 3: Run Mediator with TAK Cache Tracing
# ============================================================

# Initialize debug mediator
debug_mediator = Mediator(knowledge_base_path=KB_PATH, data_access=debug_da)
debug_repo = debug_mediator.build_repository()

# Enable detailed logging
import logging
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

# Run processing for debug patient
print(f"\n🚀 Processing patient {DEBUG_PATIENT_ID} with cache tracing...\n")
print("="*80)

# Process patient synchronously (for easier debugging)
stats, tak_cache = debug_mediator._process_patient_sync(DEBUG_PATIENT_ID, return_cache=True)

print("\n" + "="*80)
print(f"\n✅ Processing complete!")
print(f"\n📊 Output stats:")
for tak_name, count in stats.items():
    if isinstance(count, int) and count > 0:
        print(f"   - {tak_name}: {count} rows")
    elif isinstance(count, str) and count.startswith("ERROR"):
        print(f"   ❌ {tak_name}: {count}")

print(f"\n🗄️  TAK Cache Size: {len(tak_cache)} TAKs cached")


PHASE 1: Building TAK Repository


Loading TAKs: 100%|██████████| 37/37 [00:00<00:00, 279.85file/s, Patterns: INSULIN_ON_HIGH_GLUCOSE_PATTERN]     


[Validation] Running business-logic checks on TAK repository...

✅ TAK Repository Built Successfully
  Raw Concepts: 15
  Events:       4
  States:       5
  Trends:       2
  Contexts:     6
  Patterns:     5
  TOTAL TAKs:   37


🚀 Processing patient 1000 with cache tracing...





✅ Processing complete!

📊 Output stats:
   - ADMISSION: 1 rows
   - BASAL_BITZUA: 3 rows
   - DIABETES_DIAGNOSIS: 1 rows
   - GLUCOSE_MEASURE: 22 rows
   - NORMAL_GLUCOSE_IND: 19 rows
   - RELEASE: 1 rows
   - ADMISSION_EVENT: 1 rows
   - DISGLYCEMIA_EVENT: 5 rows
   - RELEASE_EVENT: 1 rows
   - BASAL_BITZUA_STATE: 3 rows
   - GLUCOSE_MEASURE_STATE: 10 rows
   - GLUCOSE_MEASURE_TREND: 15 rows
   - BASAL_BITZUA_CONTEXT: 3 rows
   - DIABETES_DIAGNOSIS_CONTEXT: 1 rows
   - HIGH_GLUCOSE_CONTEXT: 2 rows
   - GLUCOSE_MEASURE_ON_ADMISSION_PATTERN: 1 rows
   - INSULIN_ON_ADMISSION_PATTERN: 1 rows

🗄️  TAK Cache Size: 37 TAKs cached


In [16]:
# ============================================================
# Step 4: Inspect TAK Cache (Intermediate Outputs)
# ============================================================

# Query cache for each selected TAK
print(f"\n🔍 TAK-by-TAK Cache Inspection:\n")

for tak_name in DEBUG_TAKS:
    tak = debug_repo.get(tak_name)
    if not tak:
        print(f"❌ {tak_name}: TAK not found in repository")
        continue
    
    # Get from cache (not DB!)
    if tak_name not in tak_cache:
        print(f"{'='*80}")
        print(f"📌 {tak_name} ({tak.family})")
        print(f"   ⚠️  NOT IN CACHE (TAK not processed or error occurred)")
        print()
        continue
    
    df_tak_output = tak_cache[tak_name]
    
    print(f"{'='*80}")
    print(f"📌 {tak_name} ({tak.family})")
    
    # Show derived_from info
    if hasattr(tak, 'derived_from'):
        if isinstance(tak.derived_from, list):
            deps = [d['name'] for d in tak.derived_from]
            print(f"   Derived from: {deps}")
        else:
            print(f"   Derived from: {tak.derived_from}")
    
    print(f"   Cached rows: {len(df_tak_output)}")
    
    if not df_tak_output.empty:
        print(f"\n   📊 Column dtypes:")
        for col, dtype in df_tak_output.dtypes.items():
            print(f"      {col}: {dtype}")
        
        print(f"\n   📋 Sample output (first 5 rows):")
        display(df_tak_output.head(5))
        
    else:
        print(f"\n   ⚠️  EMPTY OUTPUT")
        
        # Check dependencies to diagnose why
        if hasattr(tak, 'derived_from'):
            print(f"\n   🔍 Dependency Check:")
            if isinstance(tak.derived_from, list):
                for dep in tak.derived_from:
                    dep_name = dep['name']
                    if dep_name in tak_cache:
                        dep_rows = len(tak_cache[dep_name])
                        print(f"      - {dep_name}: {dep_rows} rows")
                    else:
                        print(f"      - {dep_name}: NOT IN CACHE")
            else:
                dep_name = tak.derived_from
                if dep_name in tak_cache:
                    dep_rows = len(tak_cache[dep_name])
                    print(f"      - {dep_name}: {dep_rows} rows")
                else:
                    print(f"      - {dep_name}: NOT IN CACHE")
    
    print()


🔍 TAK-by-TAK Cache Inspection:

📌 DIABETES_DIAGNOSIS (raw-concept)
   Cached rows: 1

   📊 Column dtypes:
      PatientId: int64
      ConceptName: object
      StartDateTime: datetime64[ns]
      EndDateTime: datetime64[ns]
      Value: object
      AbstractionType: object

   📋 Sample output (first 5 rows):


,PatientId,ConceptName,StartDateTime,EndDateTime,Value,AbstractionType
0,1000,DIABETES_DIAGNOSIS,2025-01-21 09:00:00,2025-01-21 09:00:01,"(True,)",raw-concept



📌 ADMISSION (raw-concept)
   Cached rows: 1

   📊 Column dtypes:
      PatientId: int64
      ConceptName: object
      StartDateTime: datetime64[ns]
      EndDateTime: datetime64[ns]
      Value: object
      AbstractionType: object

   📋 Sample output (first 5 rows):


,PatientId,ConceptName,StartDateTime,EndDateTime,Value,AbstractionType
0,1000,ADMISSION,2025-01-21 09:00:00,2025-01-21 09:00:01,"(True,)",raw-concept



📌 GLUCOSE_MEASURE (raw-concept)
   Cached rows: 22

   📊 Column dtypes:
      PatientId: int64
      ConceptName: object
      StartDateTime: datetime64[ns]
      EndDateTime: datetime64[ns]
      Value: object
      AbstractionType: object

   📋 Sample output (first 5 rows):


...

,PatientId,ConceptName,StartDateTime,EndDateTime,Value,AbstractionType
0,1000,RELEASE_EVENT,2025-01-24 14:00:00,2025-01-24 14:00:01,True,event



📌 DIABETES_DIAGNOSIS_CONTEXT (context)
   Derived from: ['DIABETES_DIAGNOSIS']
   Cached rows: 1

   📊 Column dtypes:
      PatientId: int64
      ConceptName: object
      StartDateTime: datetime64[ns]
      EndDateTime: datetime64[ns]
      Value: object
      AbstractionType: object

   📋 Sample output (first 5 rows):


,PatientId,ConceptName,StartDateTime,EndDateTime,Value,AbstractionType
0,1000,DIABETES_DIAGNOSIS_CONTEXT,2025-01-21 09:00:01,2025-01-24 13:59:59,True,context



📌 GLUCOSE_MEASURE_ON_ADMISSION_PATTERN (local-pattern)
   Derived from: ['DIABETES_DIAGNOSIS_CONTEXT', 'GLUCOSE_MEASURE', 'ADMISSION_EVENT']
   Cached rows: 1

   📊 Column dtypes:
      PatientId: int64
      ConceptName: object
      StartDateTime: datetime64[ns]
      EndDateTime: datetime64[ns]
      Value: object
      AbstractionType: object

   📋 Sample output (first 5 rows):


,PatientId,ConceptName,StartDateTime,EndDateTime,Value,AbstractionType
0,1000,GLUCOSE_MEASURE_ON_ADMISSION_PATTERN,2025-01-21 09:00:01,2025-01-21 12:00:01,True,local-pattern


In [10]:
# ============================================================
# Step 5: Cleanup Debug DB
# ============================================================

# Close connection
debug_da.conn.close()

# Optionally delete debug DB
# DEBUG_DB_PATH.unlink()
# print(f"🗑️  Deleted debug DB: {DEBUG_DB_PATH}")

print(f"\n✅ Debug session complete!")
print(f"   Debug DB preserved at: {DEBUG_DB_PATH}")
print(f"   (Delete manually or re-run this block to recreate)")


✅ Debug session complete!
   Debug DB preserved at: backend\data\debug_mediator.db
   (Delete manually or re-run this block to recreate)
